## Agents and Tools

### Agents: Agents enable LLMs to work with tools that perform various tasks
To use agents, we require three things:
- A base LLM,
- A tool that we will be interacting with,
- An agent to control the interaction.

### In the following example, we will use the Math tool to perform some simple math operations

In [ ]:
%pip install langchain langchain_openai python-dotenv numexpr duckduckgo_search

In [ ]:
from dotenv import load_dotenv
from langchain_openai import OpenAI
load_dotenv()

In [ ]:
llm = OpenAI(
    temperature=0,
    model_name="gpt-3.5-turbo-instruct"
)

In [13]:
from langchain.chains import LLMMathChain # Chain that interprets a prompt and executes python code to do math.
from langchain.agents import Tool

In [23]:
# initialize the math tool

llm_math = LLMMathChain.from_llm(llm)

math_tool = Tool(
    name='Calculator',
    func=llm_math.run,
    description='Useful for when you need to answer questions about math.'
)

# when giving tools to LLM, we must pass as list of tools
tools = [math_tool]  # is a list of possible Tools that your agent can use

In [24]:
print(tools[0].name, "---", tools[0].description)

Calculator --- Useful for when you need to answer questions about math.


## Now create an Agent
#### Zero-shot ReAct
The Zero-shot ReAct Agent is a language generation model that can create realistic contexts even without being trained on specific data. It can be used for various tasks such as generating creative text formats, language translation, and generating different types of creative content.

In [34]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=10
)

In [36]:
zero_shot_agent.invoke({ "input" : "What is root over 25"})



> Entering new AgentExecutor chain...
 I should use a calculator to find the square root of 25
Action: Calculator
Action Input: √25
Observation: Answer: 5.0
Thought: I now know the final answer
Final Answer: 5.0

> Finished chain.


{'input': 'What is root over 25', 'output': '5.0'}

In [37]:
problem = """
        You are building a house. There are two bedrooms of 5 metres by 5 metres each and drawing cum open kitchen
        is 7 metres by 6 metres and balcony of 3 metres by 2 metres.
        What is the total area of your house?
        """
zero_shot_agent.invoke({ "input": problem })



> Entering new AgentExecutor chain...
 I need to calculate the area of each room and then add them together to get the total area.
Action: Calculator
Action Input: 5 * 5
Observation: Answer: 25
Thought: I now know the area of one bedroom.
Action: Calculator
Action Input: 7 * 6
Observation: Answer: 42
Thought: I now know the area of the drawing cum open kitchen.
Action: Calculator
Action Input: 3 * 2
Observation: Answer: 6
Thought: I now know the area of the balcony.
Action: Calculator
Action Input: 25 + 25 + 42 + 6
Observation: Answer: 98
Thought: I now know the total area of the house.
Final Answer: The total area of the house is 98 square metres.

> Finished chain.


{'input': '\n        You are building a house. There are two bedrooms of 5 metres by 5 metres each and drawing cum open kitchen\n        is 7 metres by 6 metres and balcony of 3 metres by 2 metres.\n        What is the total area of your house?\n        ',
 'output': 'The total area of the house is 98 square metres.'}

## Using mutiple Tools
In the following example, we will be using the Duck Duck Go search API as a Tool combined with the Math Tool
We will then give the Agent a problem that initiates a search first followed by a Calculation.

In [41]:
# Define the Tools

from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

search_tool = Tool.from_function(
    func=search.run,
    name="Search",
    description="useful for when you need to search the internet for information"
)

llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)

math_tool = Tool.from_function(
    func=llm_math_chain.run,
    name="Calculator",
    description="Useful for when you are asked to perform math calculations"
)

In [42]:
# Define the Agent

tools = [search_tool, math_tool]

agent = initialize_agent(
    tools,
    llm,
    agent="zero-shot-react-description",
    verbose=True
)

In [44]:
# Run the Agent with the problem

agent.invoke("""Get Microsoft Stock Price taken from Google Finance and display in both USD and INR values""")



> Entering new AgentExecutor chain...
 I should search for the current Microsoft stock price on Google Finance
Action: Search
Action Input: "Microsoft stock price"
Observation: The latest Microsoft stock prices, stock quotes, news, and MSFT history to help you invest and trade smarter. Get the latest Microsoft Corp (MSFT) stock price, earnings, news, and analysis from Morningstar. See how MSFT compares to its competitors in the software and cloud industry. Computershare, Microsoft's transfer agent, administers a direct stock purchase plan and a divident reinvestment plan for the company. To find out more about these programs you may contact Computershare directly at (800) 285-7772, Option 1, between the hours of 8 A.M. and 8 P.M. Eastern Time, Monday through Friday, and Saturday 9 A.M. and 5 P.M ... MSFT Competitors. $ Market cap P/E ratio $ Price 1d change 52-week range. View Microsoft Corporation MSFT stock quote prices, financial information, real-time forecasts, and company news 

{'input': 'Get Microsoft Stock Price taken from Google Finance and display in both USD and INR values',
 'output': 'The current Microsoft stock price is 425.52 USD or 31850.171999999995 INR.'}

## Create a custom tool

In [45]:
from langchain.tools import BaseTool
from langchain_core.output_parsers import JsonOutputParser
from langchain.tools import tool

@tool("JSON_Response", return_direct=True)
def StructuredResponseTool(question: str):
    """
    use this tool to send a prompt and get a JSON returned
    with three fields - Topic, Question_Details and Detailed_Response
    """
    json_prompt = PromptTemplate.from_template(
    """Return a JSON object with an `answer` key that answers the following question: {question}.
        The JSON object will have three fields - Topic, Question_Details and Detailed_Response"""
    )

    model = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.5)

    json_parser = JsonOutputParser()
    json_chain = json_prompt | model | json_parser
    x = json_chain.invoke({"question": question})
    return x

In [46]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

In [47]:
from langchain.agents import initialize_agent

tools = [StructuredResponseTool]

# initialize agent with tools
zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=10
)

In [49]:
zero_shot_agent.run(input="Which is the top football team in the world and how, the response should have three fields in the JSON - topic of the question, the question and the detailed response")



> Entering new AgentExecutor chain...
 I should use the JSON_Response tool to get the answer
Action: JSON_Response
Action Input: "Which is the top football team in the world and how"
Observation: {'Topic': 'Football', 'Question_Details': 'Which is the top football team in the world and how?', 'Detailed_Response': 'The top football team in the world is currently considered to be FC Barcelona from Spain. They have consistently performed well in both domestic and international competitions, winning numerous titles and setting records. Their success can be attributed to their strong team chemistry, talented players, and excellent coaching staff. Additionally, their possession-based style of play, known as tiki-taka, has been praised for its effectiveness and entertainment value. Overall, FC Barcelona has proven to be a dominant force in the world of football and is widely regarded as the top team in the world.'}


> Finished chain.


{'Topic': 'Football',
 'Question_Details': 'Which is the top football team in the world and how?',
 'Detailed_Response': 'The top football team in the world is currently considered to be FC Barcelona from Spain. They have consistently performed well in both domestic and international competitions, winning numerous titles and setting records. Their success can be attributed to their strong team chemistry, talented players, and excellent coaching staff. Additionally, their possession-based style of play, known as tiki-taka, has been praised for its effectiveness and entertainment value. Overall, FC Barcelona has proven to be a dominant force in the world of football and is widely regarded as the top team in the world.'}